<a href="https://youtu.be/A2euuevpYis" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/71px-YouTube_full-color_icon_%282017%29.svg.png" alt="Open In Colab"/></a>


# **Práctico Sistemas Recomendadores: pyreclab - Slope One**

En este práctico seguiremos utilizando [pyreclab](https://github.com/gasevi/pyreclab), con el cual estamos aprendiendo distintas técnicas de recomendación. Seguiremos usando la misma base de datos de los prácticos anteriores, para que puedan comparar los métodos y sus implementaciones. Este práctico está acompañado de un [video comentando la actividad](https://youtu.be/A2euuevpYis).

En esta oportunidad exploraremos el recomendador de Pendiente Uno o **Slope One** [1].

**Adaptado y preparado por:** Francisca Cattan 📩 fpcattan@uc.cl

Referencias 📖
------
[1] *Lemire, D., & Maclachlan, A. (2005, April). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).*


**Nombre**:  Max Guzman Aceituno

## Actividad 1 👓

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta:** Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

- Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?
- Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?
- ¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)
- ¿Qué pasaría si se agrega un nuevo rating a la base de datos?
- Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

💡 *Hint: La bibliografía todo lo puede.*

**Respuesta:**
Slope One se basa en utilizar la información de los ratings que tanto el usuario a analizar como los demás usuarios han hecho sobre un determinado item. De esta forma, es posible estimar basado en el promedio de estos usuarios que comparten ciertos ratings el rating de un nuevo item que el usuario a analizar no ha hecho (pero que el resto si).
Se trata de un recomendador basado en filtrado colaborativo, debido a que utiliza la información de otros usuarios para poder realizar la recomendación deseada. Además, según [1] se trata de un algoritmo que utiliza tanto información basada en el usuario (cantidad de usuarios que comparten ratings, por ejemplo), e información de los items en relación al rating asociado a cada uno.

Slope One selecciona en primer lugar aquellos usuarios que poseen ratings sobre los mismos items (o algunos) que el usuario a analizar, mientras también tengan rating sobre un item que este no tenga. De esta forma, se calcula para estos elementos la desviación entre dos items i y j tal que

$dev_{j,i} = \sum_{u\in S_{j,i}(\chi)}\frac{u_j - u_i}{card(S_{j,i}(\chi))}$

donde $u_i$ corresponde al rating dado por el usuario u al item i. Luego, a partir de estas desviaciónes calculadas para cada par de items i y j, para todos los usuarios, es posible realizar una predicción sobre $u_j$ dado por 

$P(u)_j = \frac{1}{card(R_j)}\sum_{i\in R_j}(dev_{j,i} + u_i)$

Lo cual puede ser simplificado a 

$P^{S1}(u)_j = \bar{u} + \frac{1}{card(R_j)}\sum_{i\in R_j}(dev_{j,i})$

Si se agrega un nuevo rating a la base de datos, se puede actualizar la matriz de desviaciones que puede almacenar el sistema. Y si llega un nuevo usuario al sistema con pocos ratings, si existe una cantidad importante de usuarios sobre esos items, entonces según el paper el sistema debería ser capaz de recomendarle satisfactoriamente elementos. 

[1] Lemire, D., & Maclachlan, A. (2005). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).





# **Configuración Inicial**

## Paso 1:
Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:


In [1]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    217      0 --:--:--  0:00:01 --:--:--   217
100 1546k  100 1546k    0     0   747k      0  0:00:02  0:00:02 --:--:--  747k


In [2]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    383      0 --:--:--  0:00:01 --:--:--   383
100  385k  100  385k    0     0   311k      0  0:00:01  0:00:01 --:--:--  311k


In [3]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    577      0 --:--:-- --:--:-- --:--:--   577
100  230k  100  230k    0     0   273k      0 --:--:-- --:--:-- --:--:--  273k


Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

## Paso 2:

Instalamos pyreclab utilizando pip.

In [4]:
!pip install pyreclab --upgrade

     |████████████████████████████████| 234 kB 21.8 MB/s 


## Paso 3:

Hacemos los imports necesarios para este práctico.

In [5]:
import pyreclab
import numpy as np
import pandas as pd

# **El dataset**

💡 *En prácticos anteriores, vimos como analizar este dataset. Puedes revisarlos en caso de dudas.*

## Paso 4:

Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [6]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# **Slope One**

## Paso 5:

Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [7]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [8]:
# Y enntrenamos
mySlopeOne.train()

## Actividad 2 👓

**Pregunta:** Explique qué hace el método `train()` en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos? 

**Respuesta:** El método train toma la nformación del dataset en base a los usuarios y sus ratings, y calcula la matriz de desviaciones para cada par de items. Además incorpora el cálculo de una matriz de cardinalidad para poder realizar el cálculo de la predicción posteriormente.

## Paso 6:

Llego la hora de predecir el rating.

In [9]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores
mySlopeOne.predict("457", "37")

3.2408759593963623

In [10]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("457", "37")

In [11]:
# Podemos comprobar las peliculas rankeadas por el usuario ID:457
# Que ciertamente ha participado activamente (¡156 items!)
train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
train_file[train_file['userid'] == 457]

,userid,itemid,rating,timestamp
37269,457,1,4,882393244
37270,457,7,4,882393278
37271,457,9,5,882393485
37272,457,11,4,882397020
37273,457,13,3,882393883
...,...,...,...,...
37420,457,1047,2,882395964
37421,457,1119,4,882398308
37422,457,1168,5,882548761
37423,457,1210,4,882549905


In [12]:
# Y también cuáles usuarios han rankeado la pelicula ID:37
train_file[train_file['itemid'] == 37]

,userid,itemid,rating,timestamp
1302,13,37,1,882397011
14851,201,37,2,884114635
19670,268,37,3,876514002
29489,363,37,2,891498510
31084,385,37,4,880013483
32996,405,37,1,885548384
62777,773,37,3,888540352


## Actividad 3 👓

Haremos un pequeño experimento para entender mejor como funciona Slope One. Gracias al ejercicio anterior, sabemos que el usuario 457 ya ha asignado el mejor rating (5 ⭐) a las dos peliculas ID:9 e ID:1168. Comparemos.

**Pregunta:** ¿Cómo se explican estos resultados?  

**Respuesta:** Estos resultados son similares (no iguales claramente) al rating entregado por el usuario 457. Esto se debe a que en base a los ratings que comparte este usuario con el resto de los usuarios, se calcula la predicción de este sobre estos items. Esto demuestra claramente que Slope One entrega una estimación bastante cercana a lo que puede ocurrir (mejor que calcular el promedio por ejemplo entre todo el resto de usuarios acerca de ese item), a pesar de que posee un error asociado tanto a que la información está basada en el resto de usuarios que es distinto a 457, como a que el usuario solo puede entregar valores enteros de rating, mientras que Slope One entrega un valor float.

In [13]:
prediction_id9 = mySlopeOne.predict("457", "9")
prediction_id1168 = mySlopeOne.predict("457", "1168")

print('Prediction for ID:9 :', prediction_id9)
print('Prediction for ID:1168 :', prediction_id1168)

Prediction for ID:9 : 4.530702114105225
Prediction for ID:1168 : 4.166153907775879


## Paso 7:

Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.



In [17]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1592', '1589', '1656', '1431', '1653']


In [18]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1431                                  Legal Deceit (1997)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## Actividad 4 👩🏻‍💻

Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN a tu elección.

**Pregunta:** ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

**Respuesta:** Si, de hecho solo una recomendación tienen en común en el top 5, y orden distinto. Y probando con otros usuarios ni siquierda poseen recomendaciones en común. Esto hace notar que este tipo de recomendador es personalizado según el usuario que se analice.

In [20]:
# Escribe el nuevo codigo aqui
reclist_slopeone = mySlopeOne.recommend("250", 10)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items según ID: ['1653', '1642', '1599', '1512', '1450', '1536', '1500', '1467', '1463', '1449']
Lista de items por nombre:


movieid
1653    Entertaining Angels: The Dorothy Day Story (1996)
1642                             Some Mother's Son (1996)
1599                        Someone Else's America (1995)
1512               World of Apu, The (Apur Sansar) (1959)
1450                               Golden Earrings (1947)
1536                                 Aiqing wansui (1994)
1500                            Santa with Muscles (1996)
1467                 Saint of Fort Washington, The (1993)
1463                                     Boys, Les (1997)
1449                               Pather Panchali (1955)
Name: title, dtype: object

## Actividad 5 👩🏻‍💻

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta:** Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

**Respuesta:** Se puede observar que las listas de recomendaciones son totalmente distintas. En este sentido, el usuario 44 podría o no compartir el gusto de las 10 películas más populares (por lo que se estima que les asigna un rating bajo), o puede ser que ya las haya visto (y por tanto el sistema no las recomienda (?)).

De todas formas, ambos sistemas son bastante rápidos para entregar predicciones, siendo most popular no personalizado, y slope one personalizado.

In [26]:
# Most popular
most_popular = pyreclab.MostPopular(dataset='u1.base',
                   dlmchar=b'\t',
                   header=False,
                   usercol=0,
                   itemcol=1,
                   ratingcol=2)

most_popular.train()

ranking = [int(r) for r in most_popular.recommend("44", 10, includeRated=False)]
print(f'Recommendation for user 44 using most popular: {ranking}')

Recommendation for user 44 using most popular: [50, 100, 181, 286, 1, 121, 300, 127, 7, 98]


In [24]:
# Escribe el nuevo codigo aqui
reclist_slopeone = mySlopeOne.recommend("44", 10)
print(f'Recommendation for user 44 using Slope One: {reclist_slopeone}')

Recommendation for user 44 using Slope One: ['1656', '1064', '1643', '1642', '1625', '1599', '1512', '1536', '1500', '1467']
